# Async and Await Primer

A quick primer on `async`/`await`. Async and await are relatively new features in Python which allow **concurent** programming. They won't make your code magically faster, but may make your code easier to read, maintain and reason about. 
You will likely hear the terms event-loop, coroutines and many other ones, they will make sens in time. 

The key thing to remember is that 
 - async-functions can call both sync and async functions.
 - sync functions can only call sync. 
 - You _must_ always `await <async functions>`


## Event loop

It's like the "One ring", there shoudl be only one. IPython (and Jupyter) usually already run one.

### Bad news
If you need to run any code that need to create and manage an event-loop, consult the docs. 
Typically you can't run a tornado app inside jupyter.

### Good news

If you don't know/don't care, all is already setup for you. 



## Example

Let's deactivate enventloop integration and try what is (usually invalid Python)

In [ ]:
%autoawait False

In [ ]:
from asyncio import sleep

In [ ]:
# does not sleep, need to be awaited
print('before sleep')
sleep(5)
print('after sleep')

In [ ]:
await sleep(5)

In [ ]:
def f():
    await sleep(5)
f()

In [ ]:
async def f():
    print('before...')
    await sleep(5)
    print('after')
### does not call f
f()

In [ ]:
await f()

... back to step beginning. 

## Autoawait

Autoawait will _attempt_ to detect async code and run it for you. There are of course limitations (bug report welcome)

In [ ]:
%autoawait True

You will note that any line that start with `%` is invalid Python and are IPython specific syntax. Those are call magics (line-magics with a single `%` sign, cell magics with a double `%%` sign)

In [ ]:
print('before')
await sleep(5)
print('after')

Top level await is now valid syntax. 

In [ ]:
tpl = 'https://anapioficeandfire.com/api/characters/{}'

In [ ]:
%%time

results = []
for i in range(1,50):
    import requests
    print('.', end='')
    r = requests.get(tpl.format(i)).json()['aliases']
    print('x', end='')
    results.append(r)
    
for r in results:
    print(r)

# Moving to asynchronous

Nothing is perfect; if you get RuntimeErrors with asyncio, you may need to restart your kernel. More during my colleagues aiohttp tutorial this Afternoon

In [ ]:
import aiohttp

In [ ]:
async with aiohttp.ClientSession() as session:
    response = await session.get(tpl.format(583))
    json = await response.json()
    print(json['aliases'])

In [ ]:
async def get_char(i, session):
        print('.', end='')
        response = await session.get(tpl.format(i))
        json = await response.json()
        print('x', end='')
        return json['aliases']

In [ ]:
async with aiohttp.ClientSession() as s:
    print(await get_char(1303, s))

In [ ]:
tasks = []

In [ ]:
import asyncio
async with aiohttp.ClientSession() as session:
    # start 
    for i in range(1,50):
        task = asyncio.ensure_future(get_char(i, session))
        tasks.append(task)
    results = await asyncio.gather(*tasks)
    for r in results:
        print(r)

## Advance Autoawait usage, Exercise

Find the documentation for autoawait, and try to make it work with another asynchronous library. For exampe try to ply with [`trio`](https://trio.readthedocs.io/en/latest/), using `trio.sleep`  and `trio.open_nursery` to get several concurent task running, pritning different message regularly and at random intervals. What happen if you use `time.sleep()` instead of `trio.sleep()` ? What hapen if you use `asyncio.sleep()` ?

In [ ]:
import trio

In [ ]:
async def every(n, message):
    for i in range(30):
        await trio.sleep(n)
        print(message)

In [ ]:
%autoawait trio

In [ ]:
async with trio.open_nursery() as nursery:
    nursery.start_soon(every, 1, 'Plic')
    nursery.start_soon(every, 2, 'Ploc')
    